In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os
import spacy



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\jaylo\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os
import spacy 
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from PIL import Image, ImageDraw
import plotnine as gg
from plotnine import *
from PIL import Image, ImageDraw, ImageChops
from sklearn.cluster import KMeans
from PIL import Image, ImageDraw
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Circle
import numpy as np
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm 

def preprocess_data(text,data_name):
    

    # Function to lemmatize text using spaCy
    def lemmatize_text(text):
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])
        # Load spaCy model
        #nlp = spacy.load("en_core_web_sm")
     # Load the data
    data = pd.read_csv(data_name)
    
    
    # Extract character names and remove from main data
    ids = data.iloc[:, 0].tolist()
    print(ids)
    main_data = data.iloc[:, 2:]

    # Identify numeric and text columns
    numeric_columns = main_data.select_dtypes(include=[np.number]).columns
    text_columns = main_data.select_dtypes(exclude=[np.number]).columns
    print(text_columns)
    # Preprocess numeric columns
    #scaler = StandardScaler()
    #imputer = SimpleImputer(strategy='mean')
    #numeric_data = pd.DataFrame(scaler.fit_transform(imputer.fit_transform(main_data[numeric_columns])), 
    #                           columns=numeric_columns)
   

    # Preprocess text columns using TfidfVectorizer and spaCy lemmatization
    if text:
        tfidf = TfidfVectorizer()  # You can adjust max_features as needed
        text_data = main_data[text_columns].fillna('')
        text_data_combined = text_data.apply(lambda x: ' '.join(x), axis=1)
        print(text_data_combined)
        #lemmatized_text = text_data_combined.apply(lemmatize_text)
        tfidf_matrix = tfidf.fit_transform(text_data_combined)
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
        processed_data = pd.concat([tfidf_df], axis=1)

    


    return [main_data,ids]

from sklearn.cluster import KMeans
from PIL import Image, ImageDraw
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Circle

def crop_to_circle(image):
    # Convert to RGBA if not already
    image = image.convert('RGBA')
    
    # Create a circular mask
    mask = Image.new('L', image.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0) + image.size, fill=255)
    
    # Get the alpha channel from the original image
    original_alpha = image.split()[3]
    
    # Combine the circular mask with the original alpha channel
    new_alpha = ImageChops.multiply(mask, original_alpha)
    
    # Create a new image with the modified alpha channel
    r, g, b, _ = image.split()
    output = Image.merge('RGBA', (r, g, b, new_alpha))
    
    return output

import numpy as np
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
def apply_force_spacing(positions, names, min_distance=1.0, iterations=50, force_strength=0.1, normalize=True):
    """
    Apply force-directed spacing to ensure elements aren't too close.

    Parameters
    ----------
    positions : np.ndarray
        Array of shape (n, d) containing the positions.
    names : list
        List of names (unused but kept for compatibility).
    min_distance : float
        Minimum allowed distance between elements.
    iterations : int
        Number of iterations for force adjustment.
    force_strength : float
        Strength of the repulsive force.
    normalize : bool
        Whether to normalize positions into [-1, 1] range before processing.
    """
    # Normalize positions first
    if normalize:
        min_vals = positions.min(axis=0)
        max_vals = positions.max(axis=0)
        ranges = np.where(max_vals - min_vals == 0, 1, max_vals - min_vals)  # prevent div by zero
        # Scale into [-1, 1]
        positions = 2 * (positions - min_vals) / ranges - 1

    for _ in tqdm(range(iterations), desc="Adjusting positions"):
        distances = squareform(pdist(positions))
        
        for i in range(len(positions)):
            too_close = np.where(distances[i] < min_distance)[0]
            too_close = too_close[too_close != i]
            
            if len(too_close) > 0:
                for j in too_close:
                    diff = positions[i] - positions[j]
                    length = np.linalg.norm(diff)
                    if length > 0:
                        diff = diff / length
                        force = force_strength * (min_distance - length)
                        positions[i] += diff * force
                        positions[j] -= diff * force
    
    return positions


def get_column(list_of_lists, column_index):
  """
  Returns a column from a list of lists.

  Args:
    list_of_lists: The list of lists.
    column_index: The index of the column to get.

  Returns:
    A list containing the elements of the specified column.
  """
  return [row[column_index] for row in list_of_lists]

def decompose_data(processed_data, id_label_data):
    image_size = .05
    image_size_ratio = image_size*50000
    def get_image(path, zoom=image_size):
        img = plt.imread(path)
        if isinstance(img, np.ndarray):
            # Convert to PIL Image
            img = Image.fromarray((img * 255).astype(np.uint8))
        img = crop_to_circle(img)
        return OffsetImage(img, zoom=zoom)

   
    
    ids = id_label_data

  
        # 2D decomposition
    tsne2 = PCA(n_components=2)
    processed_data= tsne2.fit_transform(processed_data)

    
    # Apply force-directed spacing with more reasonable parameters
    spaced_positions = apply_force_spacing(
        processed_data,
        ids,
        min_distance=0.35,  # Now works with normalized data
        iterations=1000,
        force_strength=0.01
    )

    # Clustering on the spaced positions
    kmeans_2d = KMeans(n_clusters=5, random_state=42)
    clusters_2d = kmeans_2d.fit_predict(spaced_positions)
    


    colors = [
        '#BF0D3E', 
        '#1B4D89', 
        '#006B54',  
        '#7B2CBF',  
        '#CC5500',  
        #'#116466', 
        #'#9B2226',  
        #'#354F52',
        #'#6B4423',  
        #'#693C72' 
    ]
    # Create the graph
    plt.figure()
    
    
    # Plot colored circles for each cluster
    for cluster in range(5):  # Changed to 8 clusters
        mask = clusters_2d == cluster
        plt.scatter(spaced_positions[mask, 0], spaced_positions[mask, 1], 
                   color=colors[cluster], alpha=0.7, s=image_size_ratio)

        # Add character images
    for i, character in enumerate(ids):
        try:
            img_path = os.path.join('images', f"{character}.png")
            ab = AnnotationBbox(get_image(img_path), 
                                (spaced_positions[i, 0], spaced_positions[i, 1]), 
                                frameon=False)
            plt.gca().add_artist(ab)
        except FileNotFoundError:
            print(f"Image not found for {character}")

    # Adjust plot limits to be slightly beyond [-1,1]
    x_min, x_max = spaced_positions[:, 0].min(), spaced_positions[:, 0].max()
    y_min, y_max = spaced_positions[:, 1].min(), spaced_positions[:, 1].max()

    margin = 0.3  # 10% extra space
    plt.xlim(x_min - margin, x_max + margin)
    plt.ylim(y_min - margin, y_max + margin)

    plt.suptitle('')
    plt.savefig('clustering_image.png', dpi=300)
    plt.close()

    
    print("Processing complete. Results saved to 'data-1d.csv', 'data-2d.csv' and 'TSNE_clustering_images.png'.")

df,chars = preprocess_data(False,"data.csv")
decompose_data(df,chars)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Index([], dtype='object')


Adjusting positions: 100%|██████████| 1000/1000 [00:00<00:00, 8620.62it/s]


Processing complete. Results saved to 'data-1d.csv', 'data-2d.csv' and 'TSNE_clustering_images.png'.
